# **Before Running this notebook**:

Upload the current version of the database into github:

file name: `mCPU_db_day_month_year.sqlite`


1) In MOBAXTERM `ls` into the folder you want to keep the database;

3) run `scp pioreactor@100.117.2.9:/home/pioreactor/.pioreactor/storage/pioreactor.sqlite mCPU_db_24_03_25.sqlite` replace the date.

4) Add the file to a shared Google drive folder.


# **Goals**

### 1) Download our database from github;

### 2) Get our data out of the database into a matrix in python;

### 3) make some plots with our data;



In [1]:
'https://drive.google.com/file/d/1ycfAUC_GYU2y5oRw-SxwP8lR_pRH3FfD/view?usp=drive_link'
# Download the file using its file ID
!gdown 1ycfAUC_GYU2y5oRw-SxwP8lR_pRH3FfD

Downloading...
From (original): https://drive.google.com/uc?id=1ycfAUC_GYU2y5oRw-SxwP8lR_pRH3FfD
From (redirected): https://drive.google.com/uc?id=1ycfAUC_GYU2y5oRw-SxwP8lR_pRH3FfD&confirm=t&uuid=d7222dd8-ee0b-4ad4-a41a-372684c577ee
To: /content/mCPU_db_24_03_25.sqlite
100% 736M/736M [00:08<00:00, 90.7MB/s]


In [3]:
!ls -la

total 718600
drwxr-xr-x 1 root root      4096 Mar 24 16:30 .
drwxr-xr-x 1 root root      4096 Mar 24 16:26 ..
drwxr-xr-x 4 root root      4096 Mar 20 13:31 .config
-rw-r--r-- 1 root root 735825920 Mar 24 16:28 mCPU_db_24_03_25.sqlite
drwxr-xr-x 1 root root      4096 Mar 20 13:31 sample_data


In [4]:
!pip install pony

#see: https://docs.ponyorm.org/database.html

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 kB 5.4 MB/s eta 0:00:00


In [5]:
import sqlite3

# Connect to your database file
conn = sqlite3.connect('/content/mCPU_db_24_03_25.sqlite')
cursor = conn.cursor()

# Run the integrity check
cursor.execute("PRAGMA integrity_check;")
result = cursor.fetchone()
print("Integrity check result:", result)

conn.close()

Integrity check result: ('ok',)


In [10]:
from pony.orm import Database, db_session, select, Required, PrimaryKey
import pandas as pd

db = Database()
db.bind(provider='sqlite', filename='/content/mCPU_db_24_03_25.sqlite', create_db=False)

class OD(db.Entity):
  _table_ = 'od_readings'
  pioreactor_unit = Required(str)
  timestamp = Required(str)
  od_reading = Required(float)
  experiment = Required(str)

  PrimaryKey(pioreactor_unit, timestamp)

db.generate_mapping(create_tables=False)


with db_session:
    od_data = select((o.pioreactor_unit, o.timestamp, o.od_reading, o.experiment) for o in OD)[:]


    df = pd.DataFrame(od_data, columns=['pioreactor_unit', 'timestamp', 'od_reading', 'experiment'])

df.head()


,pioreactor_unit,timestamp,od_reading,experiment
0,mCPUworker06,2025-03-13T09:44:15.398Z,0.008390,temperatureTest
1,mCPUworker06,2025-03-13T09:44:20.397Z,0.008875,temperatureTest
2,mCPUworker06,2025-03-13T09:44:25.396Z,0.008968,temperatureTest
3,mCPUworker06,2025-03-13T09:44:30.396Z,0.008998,temperatureTest
4,mCPUworker06,2025-03-13T09:44:35.397Z,0.009071,temperatureTest


In [14]:
#function to transform the timestamp into date, time, number

from datetime import datetime

def convert_timestamp(timestamp):
    dt = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%fZ")
    return dt
timestamp = "2025-03-13T09:44:15.398Z"
dt = convert_timestamp(timestamp)
print(dt)  # Output: 2025-03-13 09:44:15.398000

2025-03-13 09:44:15.398000


In [16]:
from dateutil import parser

def timePassed(referenceDate, timestamp):
    referenceDate = parser.parse(referenceDate)
    timestamp = parser.parse(timestamp)
    timePassed = timestamp.microsecond - referenceDate.microsecond
    return timePassed

referenceData = "2025-03-01T00:00:00Z"
timestamp = "2025-03-13T09:44:15.398Z"

print(timePassed(referenceData, timestamp))

398000


#try to make a plot using plotly depicting od over time perreactor (https://plotly.com/python/)

In [13]:
from datetime import timedelta
help(timedelta())

Help on timedelta object:

class timedelta(builtins.object)
 |  Difference between two datetime values.
 |  
 |  timedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
 |  
 |  All arguments are optional and default to 0.
 |  Arguments may be integers or floats, and may be positive or negative.
 |  
 |  Methods defined here:
 |  
 |  __abs__(self, /)
 |      abs(self)
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __bool__(self, /)
 |      True if self else False
 |  
 |  __divmod__(self, value, /)
 |      Return divmod(self, value).
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __floordiv__(self, value, /)
 |      Return self//value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __le__(self, value